<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/EIA/aimDPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

# Download DPR data

In [108]:
myList = pd.date_range('2013-11-20',datetime.today()- relativedelta(months=1), freq='MS').strftime("%Y/%m").tolist()
myList[-1]

'2023/11'

In [109]:
def downloadData():
    website = "https://www.eia.gov/petroleum/drilling/archive/"
    fmDPR = []  #list()  # forecastMonth of DPR
    for month in myList:
        target_url = website + month
        print(target_url)
        data = pd.read_html(target_url)
        dpr = data[1]
        dpr.columns = dpr.columns.droplevel()
        dpr['rMonth'] = dpr.columns[1]
        dpr['fMonth'] = dpr.columns[2]
        if len(dpr.columns) == 10:  dpr.drop(dpr.columns[4], axis=1, inplace=True)
        dpr.columns = ['Region', 'roMonth', 'foMonth', 'oc', 'rgMonth', 'fgMonth', 'gc', 'rMonth', 'fMonth']
        fmDPR.append(dpr)

    fmDPR = pd.concat(fmDPR).reset_index(drop=True)
    fmDPR['Month'] = pd.to_datetime(fmDPR['fMonth'])

    return fmDPR

In [118]:
ddf = downloadData()

https://www.eia.gov/petroleum/drilling/archive/2013/12


HTTPError: ignored

In [128]:
fmDPR.tail(8)

,Region,roMonth,foMonth,oc,rgMonth,fgMonth,gc,rMonth,fMonth,Month,gcn
944,Anadarko,396,392,4.0,6587,6528,59.0,November 2023,December 2023,2023-12-01,59.0
945,Appalachia,135,133,2.0,35953,35764,189.0,November 2023,December 2023,2023-12-01,189.0
946,Bakken,1270,1272,2.0,3392,3405,13.0,November 2023,December 2023,2023-12-01,13.0
947,Eagle Ford,1154,1151,3.0,7508,7457,51.0,November 2023,December 2023,2023-12-01,51.0
948,Haynesville,32,32,0.0,16559,16428,131.0,November 2023,December 2023,2023-12-01,131.0
949,Niobrara,691,692,1.0,5193,5198,5.0,November 2023,December 2023,2023-12-01,5.0
950,Permian,5976,5981,5.0,24745,24858,113.0,November 2023,December 2023,2023-12-01,113.0
951,Total,9654,9653,1.0,99937,99638,299.0,November 2023,December 2023,2023-12-01,299.0


In [127]:
fmDPR['gc'] = fmDPR['gc'].str.replace('[()]','')
fmDPR['gc'] = fmDPR['gc'].str.replace('-','0')
fmDPR['oc'] = fmDPR['oc'].str.replace('[()]','')
fmDPR['oc'] = fmDPR['oc'].str.replace('-','0')
#fmDPR['gc'] = fmDPR['gc'].as_type(float)
#df2 = fmDPR.apply(pd.to_numeric, errors='coerce').fillna(fmDPR)

fmDPR['gc'] = pd.to_numeric(fmDPR['gc'])  #, errors='coerce')
fmDPR['oc'] = pd.to_numeric(fmDPR['oc'])  #, errors='coerce')

<ipython-input-127-b2416ba98f6d>:1: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-127-b2416ba98f6d>:3: FutureWarning:

The default value of regex will change from True to False in a future version.



In [129]:
fmDPR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Region   952 non-null    object        
 1   roMonth  952 non-null    int64         
 2   foMonth  952 non-null    int64         
 3   oc       930 non-null    float64       
 4   rgMonth  952 non-null    int64         
 5   fgMonth  952 non-null    int64         
 6   gc       633 non-null    float64       
 7   rMonth   952 non-null    object        
 8   fMonth   952 non-null    object        
 9   Month    952 non-null    datetime64[ns]
 10  gcn      633 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(3)
memory usage: 81.9+ KB


In [54]:
fmDPR =fmDPR.dropna()

In [134]:
agfmDPR = fmDPR[['foMonth','fgMonth','Month']][~fmDPR['Region'].str.contains('Total')].groupby('Month').sum()
agfmDPR.tail()

,foMonth,fgMonth
Month,,
2023-08-01,9397,97972
2023-09-01,9415,98262
2023-10-01,9393,98366
2023-11-01,9553,98808
2023-12-01,9653,99638


#Read latest DPR_Data

In [14]:
dprRegions = ['Anadarko','Bakken','Eagle Ford','Niobrara','Permian','Appalachia','Haynesville']

In [15]:
dataDPR = []  # to collect DPR_Data
file = r"https://www.eia.gov/petroleum/drilling/xls/dpr-data.xlsx"
for region in dprRegions:

    data = pd.read_excel(file, sheet_name=region + ' Region', skiprows=2, usecols=[0,1,4,7])
    data.columns = ['Month', 'BH','PRo' ,'PRg']
    data['Region'] = region
    print(region)
    dataDPR.append(data)

dataDPR = pd.concat(dataDPR).reset_index(drop=True)

Anadarko
Bakken
Eagle Ford
Niobrara
Permian
Appalachia
Haynesville


In [16]:
dataDPR.head()

,Month,BH,PRo,PRg,Region
0,2007-02-01,179.8,134177.2,4053324.7,Anadarko
1,2007-03-01,165.7,137515.5,4165395.7,Anadarko
2,2007-04-01,157.0,141463.6,4095888.6,Anadarko
3,2007-05-01,170.0,144206.3,4181767.7,Anadarko
4,2007-06-01,181.0,137234.6,4192626.1,Anadarko


In [17]:
# aggregate all regions rigs, oil and gas produciton
allDPR = dataDPR[['Month','BH','PRo','PRg']].groupby('Month').sum()
allDPR

,BH,PRo,PRg
Month,,,
2007-02-01,869.8,1383719.6,19013342.8
2007-03-01,851.1,1391174.8,19296815.0
2007-04-01,854.0,1400132.3,19355381.5
2007-05-01,860.0,1395678.4,19607301.6
2007-06-01,874.0,1373247.3,19622279.1
...,...,...,...
2023-08-01,576.0,9636772.6,100386616.5
2023-09-01,557.0,9653393.4,100381820.4
2023-10-01,549.0,9657812.3,100220112.6


# Regional Analysis

In [20]:
try: value = dprR.value
except NameError: value = 'Permian'
dprR = widgets.Dropdown(options=dprRegions, value=value, description='DPR_Region:', disabled=False)

dprR

Dropdown(description='DPR_Region:', index=4, options=('Anadarko', 'Bakken', 'Eagle Ford', 'Niobrara', 'Permian…

In [135]:
myRegion = fmDPR[['Month','roMonth','foMonth','rgMonth','fgMonth','oc','gc']][fmDPR['Region']==dprR.value]   #.split()[0]]
myRegion.tail()

,Month,roMonth,foMonth,rgMonth,fgMonth,oc,gc
918,2023-08-01,5775,5764,23353,23389,11.0,36.0
926,2023-09-01,5812,5799,23632,23667,13.0,35.0
934,2023-10-01,5799,5773,23734,23745,26.0,11.0
942,2023-11-01,5931,5901,24086,24102,30.0,16.0
950,2023-12-01,5976,5981,24745,24858,5.0,113.0


In [136]:
myRegion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 5 to 950
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Month    120 non-null    datetime64[ns]
 1   roMonth  120 non-null    int64         
 2   foMonth  120 non-null    int64         
 3   rgMonth  120 non-null    int64         
 4   fgMonth  120 non-null    int64         
 5   oc       117 non-null    float64       
 6   gc       80 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 7.5 KB


In [137]:
data = dataDPR[dataDPR['Region'] == dprR.value].reset_index(drop=True)
data['Month'] = pd.to_datetime(data['Month']) #.dt.date
data[['oC','gC']] = (data[['PRo','PRg']]/1000).fillna(method='ffill').diff().round(0)
data.tail()

,Month,BH,PRo,PRg,Region,oC,gC
198,2023-08-01,324.0,5929846.1,24346476.6,Permian,28.0,225.0
199,2023-09-01,318.0,5952516.2,24494073.8,Permian,23.0,148.0
200,2023-10-01,311.0,5965837.7,24622799.6,Permian,13.0,129.0
201,2023-11-01,NaN,5975775.5,24745061.6,Permian,10.0,122.0
202,2023-12-01,NaN,5981038.0,24858200.4,Permian,5.0,113.0


In [138]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Month   203 non-null    datetime64[ns]
 1   BH      201 non-null    float64       
 2   PRo     203 non-null    float64       
 3   PRg     203 non-null    float64       
 4   Region  203 non-null    object        
 5   oC      202 non-null    float64       
 6   gC      202 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 11.2+ KB


In [139]:
plData = data.merge(myRegion, how='left', left_on='Month',right_on='Month')
plData.tail()

,Month,BH,PRo,PRg,Region,oC,gC,roMonth,foMonth,rgMonth,fgMonth,oc,gc
198,2023-08-01,324.0,5929846.1,24346476.6,Permian,28.0,225.0,5775.0,5764.0,23353.0,23389.0,11.0,36.0
199,2023-09-01,318.0,5952516.2,24494073.8,Permian,23.0,148.0,5812.0,5799.0,23632.0,23667.0,13.0,35.0
200,2023-10-01,311.0,5965837.7,24622799.6,Permian,13.0,129.0,5799.0,5773.0,23734.0,23745.0,26.0,11.0
201,2023-11-01,NaN,5975775.5,24745061.6,Permian,10.0,122.0,5931.0,5901.0,24086.0,24102.0,30.0,16.0
202,2023-12-01,NaN,5981038.0,24858200.4,Permian,5.0,113.0,5976.0,5981.0,24745.0,24858.0,5.0,113.0


In [140]:
plData['oEr'] = plData['foMonth'] - (plData['PRo']/1000).round(0)
plData['gEr'] = plData['fgMonth'] - (plData['PRg']/1000).round(0)
plData =plData.reset_index(drop=True)
plData.set_index('Month', inplace=True) #.dt.date
plData.tail()

,BH,PRo,PRg,Region,oC,gC,roMonth,foMonth,rgMonth,fgMonth,oc,gc,oEr,gEr
Month,,,,,,,,,,,,,,
2023-08-01,324.0,5929846.1,24346476.6,Permian,28.0,225.0,5775.0,5764.0,23353.0,23389.0,11.0,36.0,-166.0,-957.0
2023-09-01,318.0,5952516.2,24494073.8,Permian,23.0,148.0,5812.0,5799.0,23632.0,23667.0,13.0,35.0,-154.0,-827.0
2023-10-01,311.0,5965837.7,24622799.6,Permian,13.0,129.0,5799.0,5773.0,23734.0,23745.0,26.0,11.0,-193.0,-878.0
2023-11-01,NaN,5975775.5,24745061.6,Permian,10.0,122.0,5931.0,5901.0,24086.0,24102.0,30.0,16.0,-75.0,-643.0
2023-12-01,NaN,5981038.0,24858200.4,Permian,5.0,113.0,5976.0,5981.0,24745.0,24858.0,5.0,113.0,0.0,0.0


In [141]:
fig = go.Figure(px.scatter(title="DPR crude oil production estimates vs. latest data in %s region" % (dprR.value)))

fig.add_trace(go.Scatter(x=plData.index,
                         y=plData.PRo/1000,
                         mode='lines+markers',
                         name='latestData',
                         line=dict(color='rgb(220, 129, 46)')))

for ind in plData.index:
    fig.add_traces(go.Scatter(x=[ind - pd.DateOffset(months=1),ind],
                              y=[plData['roMonth'][ind],plData['foMonth'][ind]],
                              name=(ind - pd.DateOffset(months=1)).strftime('%b %Y'),
                              mode='lines+markers', #mode='lines+markers', mode='lines'
                              line=dict(color='rgb(100, 99, 46)')))  #line=nine_color

#fig.update_traces(connectgaps=False)
fig.update_traces(marker={'size': 4})
fig.update_layout(showlegend=False, xaxis={'title':'month'},
                                    yaxis=dict(title='Mbpd',tickformat=',.0f')) # 'ticksuffix': 'Mbpd','range':[1000,6000],

fig.show()

In [142]:
fig = go.Figure(px.scatter(title="DPR crude oil production forecast in %s region" % (dprR.value)))
fig.add_trace(go.Scatter(x=plData.index, y=plData.foMonth, mode='markers',name='PredictError',line=dict(color='rgb(20, 129, 146)')))
fig.add_trace(go.Scatter(x=plData.index, y=plData.PRo/1000, mode='lines+markers',name='latestData',line=dict(color='rgb(220, 129, 46)')))
fig.update_layout(xaxis_title='month', yaxis=dict(title='Mbpd',tickformat=',.0f'))  #,legend={'traceorder': 'reversed'})
fig.show()

In [143]:
fig = go.Figure(px.scatter(title="DPR crude oil production forecast errors %s region" % (dprR.value)))
fig.add_trace(go.Scatter(x=plData.index, y=plData.oEr, mode='lines+markers',name='PredictError',line=dict(color='rgb(220, 129, 46)')))
fig.update_layout(xaxis_title='month', yaxis=dict(title='Mbpd',tickformat=',.0f'))  #,legend={'traceorder': 'reversed'})
fig.show()

In [154]:
plData['oEr'].plot(title='Forecasting errors histogram for oils estimates in the %s region' %(dprR.value),
                   kind='hist', nbins=25,
                   color_discrete_map={'oEr':'rgb(220, 129, 46)'}).update_layout(showlegend=False, xaxis=dict(title='MMcfpd',tickformat=',.0f')).update_traces(marker_line_width=1,marker_line_color="white")

In [144]:
fig = go.Figure(px.scatter(title="DPR natural gas production estimates vs. latest data in %s region" % (dprR.value)))

fig.add_trace(go.Scatter(x=plData.index,
                         y=plData.PRg/1000,
                         mode='lines+markers',
                         name='latestData',
                         line=dict(color='rgb(82, 189, 255)')))


for ind in plData.index:
    fig.add_traces(go.Scatter(x=[ind - pd.DateOffset(months=1),ind],
                              y=[plData['rgMonth'][ind],plData['fgMonth'][ind]],
                              name=(ind - pd.DateOffset(months=1)).strftime('%b %Y'),
                              mode='lines+markers', #mode='lines+markers', mode='lines'
                              line=dict(color='rgb(100, 99, 146)')))  #line=nine_color

#fig.update_traces(connectgaps=False)
fig.update_traces(marker={'size': 4})
fig.update_layout(showlegend=False, xaxis={'title':'month'},
                                    yaxis={'title':'MMcfpd','tickformat':',.0f'}) # 'ticksuffix': 'Mbpd','range':[1000,6000],

fig.show()

In [145]:
fig = go.Figure(px.scatter(title="DPR natural gas production forecast errors %s" % (dprR.value)))
fig.add_trace(go.Scatter(x=plData.index, y=plData.fgMonth, mode='markers',name='PredictedValue',line=dict(color='rgb(120, 129, 146)')))
fig.add_trace(go.Scatter(x=plData.index, y=plData.PRg/1000, mode='lines+markers',name='latestData',line=dict(color='rgb(82, 189, 255)')))
fig.update_layout(xaxis_title='month', yaxis_title='MMcfpd', yaxis=dict(ticksuffix='', tickformat=',.0f'))  #,legend={'traceorder': 'reversed'})
fig.show()

In [146]:
fig = go.Figure(px.scatter(title="DPR natural gas production forecast errors %s" % (dprR.value)))
fig.add_trace(go.Scatter(x=plData.index, y=plData.gEr, mode='lines+markers',name='PredictError',line=dict(color='rgb(82, 189, 255)')))
fig.update_layout(xaxis_title='month', yaxis=dict(title='MMcfpd',tickformat=',.0f',ticksuffix=''))  #legend={'traceorder': 'reversed'})
fig.show()

In [147]:
plData['gEr'].plot(title='Forecasting errors histogram for natural gas estimates in the %s region' %(dprR.value),
                   kind='hist', nbins=25,
                   color_discrete_map={'gEr':'rgb(82, 189, 255)'}).update_layout(showlegend=False, xaxis=dict(title='MMcfpd',tickformat=',.0f')).update_traces(marker_line_width=1,marker_line_color="white")

In [148]:
plData['year']=pd.to_datetime(plData.index)
plData['year']=plData['year'].dt.year
plData.tail(2)

,BH,PRo,PRg,Region,oC,gC,roMonth,foMonth,rgMonth,fgMonth,oc,gc,oEr,gEr,year
Month,,,,,,,,,,,,,,,
2023-11-01,NaN,5975775.5,24745061.6,Permian,10.0,122.0,5931.0,5901.0,24086.0,24102.0,30.0,16.0,-75.0,-643.0,2023
2023-12-01,NaN,5981038.0,24858200.4,Permian,5.0,113.0,5976.0,5981.0,24745.0,24858.0,5.0,113.0,0.0,0.0,2023


In [149]:
plData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 203 entries, 2007-02-01 to 2023-12-01
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BH       201 non-null    float64
 1   PRo      203 non-null    float64
 2   PRg      203 non-null    float64
 3   Region   203 non-null    object 
 4   oC       202 non-null    float64
 5   gC       202 non-null    float64
 6   roMonth  120 non-null    float64
 7   foMonth  120 non-null    float64
 8   rgMonth  120 non-null    float64
 9   fgMonth  120 non-null    float64
 10  oc       117 non-null    float64
 11  gc       80 non-null     float64
 12  oEr      120 non-null    float64
 13  gEr      120 non-null    float64
 14  year     203 non-null    int64  
dtypes: float64(13), int64(1), object(1)
memory usage: 33.5+ KB


In [150]:
fig = px.scatter(x=plData.gc, y=plData.gC, color=plData['year'], trendline="ols", trendline_color_override="black", color_continuous_scale=px.colors.sequential.Bluered)

fig.update_traces(marker={'size': 6})
fig.update_layout(showlegend=False, title="DPR natural gas production change estimates vs. actual change in %s region" % (dprR.value),
                  xaxis={'title':'estimated change MMcfpd','range':[-300,300]},
                  yaxis={'title':'actual change MMcfpd','tickformat':',.0f','range':[-600,600]}) # 'ticksuffix': 'Mbpd','range':[1000,6000],
resultsGas = px.get_trendline_results(fig)
fig.show()

In [151]:
resultsGas.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.8113
Date:                Sat, 16 Dec 2023   Prob (F-statistic):              0.370
Time:                        04:13:33   Log-Likelihood:                -630.21
No. Observations:                  80   AIC:                             1264.
Df Residuals:                      78   BIC:                             1269.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         80.1544    115.282      0.695      0.489    -149.354     309.663
x1             0.8530      0.947      0.901      0.370      -1.032       2.738
==============================================================================
Omnibus:                       35.195   Durbin-Watson:                   2.073
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              694.198
Skew:                          -0.447   Prob(JB):                    1.81e-151
Kurtosis:                      17.404   Cond. No.                         194.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [152]:
fig = px.scatter(x=plData.oc, y=plData.oC, color=plData['year'], trendline="ols", trendline_color_override="black", color_continuous_scale=px.colors.sequential.Bluered)


fig.update_traces(marker={'size': 6})
fig.update_layout(showlegend=False, title="DPR oil production change estimates vs. actual change in %s region" % (dprR.value),
                  xaxis={'title':'estimated change MMcfpd','range':[-100,100]},
                  yaxis={'title':'actual change MMcfpd','tickformat':',.0f','range':[-200,200]}),  # 'ticksuffix': 'Mbpd'
resultsOil = px.get_trendline_results(fig)
fig.show()